## Sample Surprise Usage - Recommendation Model

In [1]:
import numpy as np
import pandas as pd

In [2]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
ratings_df = pd.read_csv('./ml-100k/u.data', sep='\t', names=names)
ratings_df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
print(ratings_df.describe())
print("")
nusers = ratings_df["user_id"].nunique()
nproducts = ratings_df["item_id"].nunique()
print(str(nusers)+ " number of unique users")
print(str(nproducts)+" number of unique items")

            user_id        item_id         rating     timestamp
count  100000.00000  100000.000000  100000.000000  1.000000e+05
mean      462.48475     425.530130       3.529860  8.835289e+08
std       266.61442     330.798356       1.125674  5.343856e+06
min         1.00000       1.000000       1.000000  8.747247e+08
25%       254.00000     175.000000       3.000000  8.794487e+08
50%       447.00000     322.000000       4.000000  8.828269e+08
75%       682.00000     631.000000       4.000000  8.882600e+08
max       943.00000    1682.000000       5.000000  8.932866e+08

943 number of unique users
1682 number of unique items


In [4]:
ratings = np.zeros((nusers , nproducts))

In [5]:

# create a nusers X nitems matrix and fill in the ratings values
for row in ratings_df.itertuples():    
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [6]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
#         print(test_ratings)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [7]:
train, test = train_test_split(ratings)
print(train.shape)
print(test.shape)

(943, 1682)
(943, 1682)


In [11]:
import sklearn.metrics.pairwise

sim = sklearn.metrics.pairwise.cosine_similarity(train)

In [29]:
sim.shape

(943, 943)

In [19]:
sim.dot(ratings).shape

(943, 1682)

In [30]:
# find top k users
k = 40
topk_users = np.argsort(sim)[::-1][:,:k]
topk_users.shape

(943, 40)

In [34]:
sim[1,:][topk_users].shape

(943, 40)

In [41]:
pred = np.zeros(ratings.shape)

for i in range(1):
    top_k_users = np.argsort(sim)[::-1][:,:k]
    for j in range(1):
        print(sim[i, :][top_k_users])
#         pred[i, j] = sim[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
#         pred[i, j] /= np.sum(np.abs(sim[i, :][top_k_users]))

[[0.04246507 0.04857668 0.03659149 ... 0.04281694 0.05590607 0.06953194]
 [0.0112352  0.16323997 0.04710228 ... 0.08591283 0.07015717 0.0483224 ]
 [0.02958247 0.04109062 0.04934126 ... 0.20348051 0.03136474 0.06485375]
 ...
 [0.08756336 0.19213031 0.17929848 ... 0.36943377 0.16917381 0.21963097]
 [0.1168792  0.15026322 0.09648089 ... 0.10065468 0.14273825 0.1498689 ]
 [0.         0.         0.         ... 0.02112611 0.02317461 0.02340399]]


In [43]:
top_k_users = np.argsort(sim)[::-1][:,:k]

In [44]:
top_k_users.shape

(943, 40)

In [56]:
ratings[1, [top_k_users]].shape

(1, 943, 40)

In [14]:
def cosine_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [15]:

user_cosine_similarity = cosine_similarity(train, kind='user')
item_cosine_similarity = cosine_similarity(train, kind='item')
print(item_cosine_similarity[:4, :4])

[[1.         0.41099516 0.32290243 0.45816468]
 [0.41099516 1.         0.26223334 0.49585896]
 [0.32290243 0.26223334 1.         0.2949136 ]
 [0.45816468 0.49585896 0.2949136  1.        ]]


In [16]:
user_cosine_similarity

array([[1.        , 0.15861357, 0.03762763, ..., 0.15518078, 0.17000039,
        0.37819237],
       [0.15861357, 1.        , 0.08459062, ..., 0.06878963, 0.13714026,
        0.11044833],
       [0.03762763, 0.08459062, 1.        , ..., 0.02938477, 0.10251852,
        0.01425584],
       ...,
       [0.15518078, 0.06878963, 0.02938477, ..., 1.        , 0.11296811,
        0.08159914],
       [0.17000039, 0.13714026, 0.10251852, ..., 0.11296811, 1.        ,
        0.1735517 ],
       [0.37819237, 0.11044833, 0.01425584, ..., 0.08159914, 0.1735517 ,
        1.        ]])

### Surprise package usage for recommendations

In [1]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [2]:
data = Dataset.load_builtin('ml-100k')

algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9489  0.9457  0.9283  0.9296  0.9336  0.9372  0.0085  
MAE (testset)     0.7484  0.7428  0.7329  0.7320  0.7372  0.7387  0.0062  
Fit time          4.35    4.47    4.37    4.50    4.34    4.41    0.06    
Test time         0.17    0.11    0.15    0.16    0.16    0.15    0.02    


{'fit_time': (4.348273038864136,
  4.46734619140625,
  4.374805927276611,
  4.495290040969849,
  4.3423378467559814),
 'test_mae': array([0.74842831, 0.74284802, 0.7328509 , 0.73198626, 0.73716462]),
 'test_rmse': array([0.94885058, 0.94573355, 0.92832324, 0.92963279, 0.93355875]),
 'test_time': (0.16629767417907715,
  0.11486601829528809,
  0.1534280776977539,
  0.15606284141540527,
  0.15705299377441406)}

In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.25)

In [3]:
algo = SVDpp()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9205  0.9105  0.9267  0.9195  0.9149  0.9184  0.0055  
MAE (testset)     0.7202  0.7131  0.7262  0.7204  0.7186  0.7197  0.0042  
Fit time          160.64  159.11  160.18  159.91  160.55  160.08  0.55    
Test time         2.86    2.86    3.48    3.14    2.95    3.06    0.23    


{'fit_time': (160.6398410797119,
  159.11276173591614,
  160.17634797096252,
  159.91279578208923,
  160.5505018234253),
 'test_mae': array([0.72022661, 0.71306479, 0.72619606, 0.72041391, 0.71855737]),
 'test_rmse': array([0.92051117, 0.9105373 , 0.9267286 , 0.91950087, 0.91485181]),
 'test_time': (2.8552191257476807,
  2.8628718852996826,
  3.478797197341919,
  3.141758918762207,
  2.947489023208618)}

In [4]:
algo = NMF()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9715  0.9677  0.9632  0.9596  0.9578  0.9640  0.0051  
MAE (testset)     0.7638  0.7620  0.7558  0.7558  0.7525  0.7580  0.0042  
Fit time          5.02    4.54    4.67    4.71    4.75    4.74    0.16    
Test time         0.13    0.14    0.14    0.14    0.14    0.14    0.00    


{'fit_time': (5.024583101272583,
  4.540709018707275,
  4.670447826385498,
  4.714916706085205,
  4.754637718200684),
 'test_mae': array([0.76379854, 0.76195173, 0.75575786, 0.75582593, 0.75249666]),
 'test_rmse': array([0.97149799, 0.96767898, 0.96319345, 0.95962072, 0.95777352]),
 'test_time': (0.13350415229797363,
  0.14029693603515625,
  0.13926291465759277,
  0.13964605331420898,
  0.14038395881652832)}